In [ ]:
!pip install python-docx PyPDF2 antiword

import re
import string
from collections import Counter
import requests
from google.colab import files
import os

# For PDF files
try:
    import PyPDF2
except:
    !pip install PyPDF2
    import PyPDF2

# For DOCX files
try:
    import docx
except:
    !pip install python-docx
    import docx


In [ ]:
def read_txt_file(file_path):
    """Read text from .txt file"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding='latin-1') as file:
            return file.read()

def read_docx_file(file_path):
    """Read text from .docx file"""
    doc = docx.Document(file_path)
    text = []
    for paragraph in doc.paragraphs:
        if paragraph.text.strip():
            text.append(paragraph.text)
    return '\n'.join(text)

def read_pdf_file(file_path):
    """Read text from .pdf file"""
    text = ""
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

def read_doc_file(file_path):
    """Read text from .doc file using antiword"""
    try:
        # Use antiword command-line tool for .doc files
        !apt-get install antiword > /dev/null 2>&1
        text = !antiword -w 0 "{file_path}" 2>/dev/null
        return '\n'.join(text)
    except:
        return "Unable to read .doc file content. Please convert to .docx or .txt format."

def clean_text(text):
    """Clean and preprocess text for analysis"""
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove numbers and extra whitespace
    text = re.sub(r'\d+', '', text)
    text = ' '.join(text.split())
    return text

def analyze_text(text):
    """Analyze text and return statistics"""
    if not text or text.startswith("Unable to read") or text.startswith("Error"):
        return {
            'lines': 0,
            'words': 0,
            'characters': 0,
            'top_5_words': [],
            'error': text
        }

    # Basic statistics
    lines = [line for line in text.split('\n') if line.strip()]
    words = text.split()
    characters = len(text.replace('\n', '').replace('\r', '').replace(' ', ''))

    # Clean text for word frequency analysis
    cleaned_text = clean_text(text)
    word_list = [word for word in cleaned_text.split() if len(word) > 1]

    # Word frequency
    word_freq = Counter(word_list)
    top_5_words = word_freq.most_common(5)

    return {
        'lines': len(lines),
        'words': len(words),
        'characters': characters,
        'top_5_words': top_5_words,
        'error': None
    }

def process_file(file_path):
    """Process file based on its extension"""
    file_extension = file_path.lower().split('.')[-1]

    print(f"Processing {file_extension.upper()} file...")

    if file_extension == 'txt':
        return read_txt_file(file_path)
    elif file_extension == 'docx':
        return read_docx_file(file_path)
    elif file_extension == 'pdf':
        return read_pdf_file(file_path)
    elif file_extension == 'doc':
        return read_doc_file(file_path)
    else:
        return f"Unsupported file format: {file_extension}"

def display_results(stats, filename):
    """Display analysis results in a formatted way"""
    print("=" * 60)
    print(f"📊 TEXT ANALYSIS REPORT: {filename}")
    print("=" * 60)

    if stats['error']:
        print(f"❌ Error: {stats['error']}")
        print("=" * 60)
        return

    print(f"📈 Basic Statistics:")
    print(f"   • Number of lines: {stats['lines']:,}")
    print(f"   • Number of words: {stats['words']:,}")
    print(f"   • Number of characters: {stats['characters']:,}")

    if stats['top_5_words']:
        print(f"\n🏆 Top 5 Most Frequent Words:")
        for i, (word, count) in enumerate(stats['top_5_words'], 1):
            print(f"   {i}. '{word}' - {count} occurrence(s)")
    else:
        print(f"\nℹ️  No words found for frequency analysis")

    print("=" * 60)

# Main function to run the text analyzer
def text_analyzer():
    """Main function to run the text analyzer"""
    print("🎯 TEXT ANALYZER PROJECT")
    print("=" * 40)
    print("Supported formats: .txt, .docx, .pdf")
    print(".doc files have limited support")
    print("=" * 40)

    print("\nChoose an option:")
    print("1. 📁 Upload a file")
    print("2. ⌨️ Enter text manually")
    print("3. 📚 Analyze multiple files")

    choice = input("\nEnter your choice (1-3): ").strip()

    if choice == '1':
        # File upload option
        print("\nPlease upload your file:")
        uploaded = files.upload()

        if not uploaded:
            print("❌ No file uploaded!")
            return

        filename = list(uploaded.keys())[0]

        try:
            # Save uploaded file temporarily
            with open(filename, 'wb') as f:
                f.write(uploaded[filename])

            # Process the file
            text = process_file(filename)
            stats = analyze_text(text)
            display_results(stats, filename)

        except Exception as e:
            print(f"❌ Error processing file: {str(e)}")

    elif choice == '2':
        # Manual text input
        print("\nEnter your text (press Enter twice to finish):")
        lines = []
        while True:
            line = input()
            if line == '' and lines and lines[-1] == '':
                break
            lines.append(line)

        text = '\n'.join(lines[:-1])  # Remove the last empty line
        stats = analyze_text(text)
        display_results(stats, "Manual Input")

    elif choice == '3':
        # Analyze multiple files
        analyze_multiple_files()

    else:
        print("❌ Invalid choice!")

def analyze_multiple_files():
    """Analyze multiple files at once"""
    print("\n📚 Upload multiple files for analysis:")
    uploaded_files = files.upload()

    if not uploaded_files:
        print("❌ No files uploaded!")
        return

    results = {}
    for filename, content in uploaded_files.items():
        print(f"\nProcessing {filename}...")
        try:
            # Save uploaded file temporarily
            with open(filename, 'wb') as f:
                f.write(content)

            # Process the file
            text = process_file(filename)
            stats = analyze_text(text)
            results[filename] = stats

        except Exception as e:
            print(f"❌ Error processing {filename}: {str(e)}")
            results[filename] = {'error': str(e)}

    # Display all results
    print("\n" + "=" * 60)
    print("📊 MULTIPLE FILES ANALYSIS SUMMARY")
    print("=" * 60)

    for filename, stats in results.items():
        display_results(stats, filename)

# Quick analyzer function for direct use
def quick_analyze():
    """Quick analysis function for immediate use"""
    uploaded = files.upload()
    if not uploaded:
        print("❌ No file uploaded!")
        return

    filename = list(uploaded.keys())[0]
    with open(filename, 'wb') as f:
        f.write(uploaded[filename])

    text = process_file(filename)
    stats = analyze_text(text)
    display_results(stats, filename)

# Run the text analyzer
if __name__ == "__main__":
    text_analyzer()